In [1]:
import pandas as pd

In [2]:
with pd.HDFStore('evo/Y1.h5',  mode='r') as store:
    Y = store.select('Y')
    
with pd.HDFStore('evo/X1.h5',  mode='r') as store:
    X = store.select('X')    

In [4]:
X['InclusionBodiesProbPos'] = (X['InclusionBodiesProbability']>=0)*X['InclusionBodiesProbability']
X['InclusionBodiesProbNeg'] = (X['InclusionBodiesProbability']<0)*X['InclusionBodiesProbability']

def location_type_numeric(x1):
    
    if x1 == '0':
        return 1
    elif x1 == 'SignalP_gram_plus_S':
        return 2
    elif x1 == 'SignalP_gram_minus_S':
        return 3
    elif x1 == 'T4SEpre_bpbAac_T4SE':
        return 4
    elif x1 == 'hmmsearch_Tat':
        return 5
    elif x1 == 'T3_MM_T3SS':
        return 6
    elif x1 == 'SignalP_gram_plus_S':
        return 7
    elif x1 == 'hmmsearch_T6S':
        return 8
    elif x1 == 'hmmsearch_T1S':
        return 9
    else:
        raise Exception(x1)
        
X['loc_type_numeric'] = X['loc_type'].apply(location_type_numeric)


In [5]:
Y.sort_values('cluster', inplace=True)
X.sort_values('cluster_is', inplace=True)

In [6]:
Y.drop('cluster', inplace=True,axis=1)
X.drop(['cluster_is', 'InclusionBodiesProbability', 'loc_type' ], inplace=True,axis=1)

In [7]:
labels_on_sums = Y.sum(axis=0)
labels_on_sums.sort_values(ascending=False, inplace=True)

In [8]:
x1 = labels_on_sums > 1000

In [14]:
x1

go_9401      True
go_45454     True
go_105       True
go_6265      True
go_9231      True
go_9103      True
go_6284      True
go_6189      True
go_6098      True
go_6207      True
go_44780     True
go_6094      True
go_9405      True
go_15991     True
go_9234      True
go_9102      True
go_15628     True
go_9245      True
go_9432      True
go_70526     True
go_917       True
go_6535      True
go_162       True
go_42128     True
go_15979     True
go_6614      True
go_9088      True
go_19277     True
go_46854     True
go_6108      True
            ...  
go_16973    False
go_6948     False
go_7172     False
go_32007    False
go_42273    False
go_7131     False
go_55       False
go_16559    False
go_30150    False
go_244      False
go_48478    False
go_45122    False
go_48484    False
go_71929    False
go_30041    False
go_7128     False
go_2084     False
go_6689     False
go_52007    False
go_7304     False
go_31424    False
go_60271    False
go_6654     False
go_19510    False
go_71596  

In [9]:
valid_labels = labels_on_sums[x1]

In [15]:
valid_labels

go_9401       168838
go_45454      134060
go_105         83332
go_6265        69816
go_9231        66419
go_9103        62113
go_6284        61957
go_6189        56105
go_6098        49534
go_6207        46057
go_44780       43153
go_6094        42789
go_9405        39203
go_15991       39192
go_9234        32044
go_9102        31701
go_15628       30616
go_9245        29815
go_9432        28998
go_70526       28228
go_917         27548
go_6535        27144
go_162         25012
go_42128       24790
go_15979       24317
go_6614        24168
go_9088        22494
go_19277       22091
go_46854       21729
go_6108        21230
               ...  
go_9436         1709
go_10501        1669
go_51391        1653
go_9271         1629
go_46411        1604
go_6102         1598
go_1901106      1595
go_6212         1592
go_46392        1570
go_70395        1559
go_9772         1541
go_15757        1538
go_48473        1459
go_6809         1408
go_6120         1312
go_9758         1305
go_71918     

In [11]:
columsn_to_drop = [col for col in Y.columns if  col not in valid_labels]

In [12]:
Y.drop(columsn_to_drop, inplace=True,axis=1)

In [31]:
Y.shape

(2528843, 245)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [33]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [13]:
alg = xgb.XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=5,
                        min_child_weight=3, gamma=0.2, subsample=0.6, colsample_bytree=1.0,
                        objective='binary:logistic', nthread=4, scale_pos_weight=1, seed=27)

In [14]:
alg.fit(X_train, y_train['go_9401'], eval_metric='auc')

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1.0, gamma=0.2,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=3, missing=None, n_estimators=140, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27,
              silent=None, subsample=0.6, verbosity=1)

In [15]:
predictions = alg.predict(X_test)
pred_proba = alg.predict_proba(X_test)[:, 1]

In [14]:
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from imblearn.metrics import classification_report_imbalanced




/home/ubuntu/.local/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [18]:
print("acc : %.4g" % metrics.accuracy_score(y_test['go_9401'], predictions))
print("roc_auc %f" % metrics.roc_auc_score(y_test['go_9401'], pred_proba))
print("F1 Score : %f" % metrics.f1_score(y_test['go_9401'], predictions))

acc : 0.9851
AUC 0.991573
F1 Score : 0.880308


In [25]:
for label, count in labels_on_sums.items():
    alg = xgb.XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=5,
                        min_child_weight=3, gamma=0.2, subsample=0.6, colsample_bytree=1.0,
                        objective='binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
    
    alg.fit(X_train, y_train[label], eval_metric='auc')
    predictions = alg.predict(X_test)
    print('****************************************************')
    print('label {0} with rows {1}'.format(label, count))
    
    print("acc : %.4g" % metrics.accuracy_score(y_test[label], predictions))
    print("AUC %f" % metrics.roc_auc_score(y_test[label], pred_proba))
    print("F1 Score : %f" % metrics.f1_score(y_test[label], predictions))
    print(classification_report_imbalanced(y_test[label], predictions))

    print('****************************************************')
    
    

****************************************************
label go_9401 with rows 168838
acc : 0.9851
AUC 0.991573
F1 Score : 0.880308
                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      1.00      0.82      0.99      0.90      0.83    707882
          1       0.95      0.82      1.00      0.88      0.90      0.80     50771

avg / total       0.98      0.99      0.83      0.98      0.90      0.83    758653

****************************************************
****************************************************
label go_45454 with rows 134060
acc : 0.9827
AUC 0.583986
F1 Score : 0.815040
                   pre       rec       spe        f1       geo       iba       sup

          0       0.98      1.00      0.72      0.99      0.85      0.74    718273
          1       0.94      0.72      1.00      0.82      0.85      0.70     40380

avg / total       0.98      0.98      0.73      0.98      0.85      0.73    758653

****************

KeyboardInterrupt: 

In [35]:
#chain classifier
labels_order = []
for label, count in valid_labels.items():
    
    my_index = y_train.columns.tolist().index(label)
    labels_order.append(my_index)
    

In [ ]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
# initialize classifier chains multi-label classifier
# with a gaussian naive bayes base classifier use any other classifier if u wish
#classifier = ClassifierChain(GaussianNB())
#classifier = ClassifierChain(DecisionTreeClassifier())

print(1)
algo = xgb.XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=5,
                        min_child_weight=3, gamma=0.2, subsample=0.6, colsample_bytree=1.0,
                        objective='binary:logistic', nthread=4, scale_pos_weight=1, seed=27)

print(2)
classifier = ClassifierChain(algo, order=labels_order)
#classifier = ClassifierChain(algo)

print(3)
# train
classifier.fit(X_train, y_train)
print(4)

1
2
3


In [33]:
X_train.ndim,y_train.ndim

(2, 2)

In [ ]:
# predict
predictions = classifier.predict(x)
accuracy_score(y,predictions)